<a href="https://colab.research.google.com/github/abrarashraf9/Newsapp/blob/main/NLP%20-%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
lakshmi25npathi_imdb_dataset_of_50k_movie_reviews_path = kagglehub.dataset_download('lakshmi25npathi/imdb-dataset-of-50k-movie-reviews')

print('Data source import complete.')


Data source import complete.


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import nltk
import string
from nltk.corpus import stopwords
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [5]:
df.duplicated().sum()

np.int64(418)

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.shape

(49582, 2)

In [9]:
df.count()

,0
review,49582
sentiment,49582


In [10]:
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

<ipython-input-10-0554edc9e970>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [11]:
df['review']=df['review'].str.lower()

In [12]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. <br /><br />the...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


In [13]:
from tqdm import tqdm
tqdm.pandas()

def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub('', text)

df['review'] = df['review'].progress_apply(lambda x: remove_html_tags(x))
df.head()

100%|██████████| 49582/49582 [00:00<00:00, 194955.52it/s]


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


In [14]:
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)
df['review']=df['review'].progress_apply(lambda x : remove_url(x))
df.head()

100%|██████████| 49582/49582 [00:01<00:00, 43185.64it/s]


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


In [15]:
def remove_digits(text):
    return re.sub(r'\d+', '', text)
df['review']=df['review'].progress_apply(lambda x : remove_digits(x))
df.head()

100%|██████████| 49582/49582 [00:02<00:00, 19889.68it/s]


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production. the filming tec...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there's a family where a little boy ...,0
4,"petter mattei's ""love in the time of money"" is...",1


In [16]:
exclude=string.punctuation
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
def remove_punc(text):
    for char in exclude:
        text=text.replace(char,'')
    return text
df['review']=df['review'].progress_apply(lambda x : remove_punc(x))
df.head()

100%|██████████| 49582/49582 [00:01<00:00, 25782.39it/s]


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically theres a family where a little boy j...,0
4,petter matteis love in the time of money is a ...,1


In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def remove_stopwords(text):
    StopWords = set(stopwords.words('english'))
    filtered_text = ' '.join(word for word in text.split() if word.lower() not in StopWords)
    return filtered_text
df['review']=df['review'].progress_apply(lambda x : remove_stopwords(x))
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 49582/49582 [00:09<00:00, 5135.65it/s]


,review,sentiment
0,one reviewers mentioned watching oz episode yo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1


In [19]:
def remove_special_characters(text):
    text = re.sub(r'[^\w]', ' ', text)
    return text
df['review']=df['review'].progress_apply(lambda x : remove_special_characters(x))
df.head()

100%|██████████| 49582/49582 [00:02<00:00, 24195.91it/s]


,review,sentiment
0,one reviewers mentioned watching oz episode yo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1


In [20]:
def remove_newline(text):
    text = re.sub(r'\n', ' ', text)
    return text
df['review']=df['review'].progress_apply(lambda x : remove_newline(x))
df.head()

100%|██████████| 49582/49582 [00:00<00:00, 426321.95it/s]


,review,sentiment
0,one reviewers mentioned watching oz episode yo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1


In [21]:
# How to remove letters containing numbers? ex: aa34
def remove_letters_with_numbers(text):
    text = re.sub(r'\w*\d\w*', '', text)
    return text

df['review']=df['review'].progress_apply(lambda x : remove_letters_with_numbers(x))
df.head()

100%|██████████| 49582/49582 [00:06<00:00, 7359.31it/s]


,review,sentiment
0,one reviewers mentioned watching oz episode yo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1


In [22]:
ps=PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [23]:
le=WordNetLemmatizer()
def lemm_words(text):
    return " ".join([le.lemmatize(word) for word in text.split()])

In [24]:
df.head()

,review,sentiment
0,one reviewers mentioned watching oz episode yo...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1


In [25]:
X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
vectorizer =TfidfVectorizer(ngram_range=(1, 2), max_features=15000, stop_words='english')

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [35]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [40]:
y_pred = model.predict(X_test_tfidf)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8832308157708985
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      4939
           1       0.88      0.89      0.88      4978

    accuracy                           0.88      9917
   macro avg       0.88      0.88      0.88      9917
weighted avg       0.88      0.88      0.88      9917



In [37]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_train_tfidf, y_train)


LinearSVC()

In [39]:
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      4939
           1       0.88      0.89      0.88      4978

    accuracy                           0.88      9917
   macro avg       0.88      0.88      0.88      9917
weighted avg       0.88      0.88      0.88      9917

